In [33]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [34]:
source = requests.get('https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin').text
soup=BeautifulSoup(source,'lxml')
from IPython.display import display_html
tab = str(soup.table)
display_html(tab,raw=True)
dfs = pd.read_html(tab)
df=dfs[0]
df.drop ("Map", axis =1, inplace= True)
df

Borough
 
 Population 31 March 2010 
 
 Area in km² 
 
 Density per km² 
 
 Map
 
 
 Charlottenburg-Wilmersdorf 
 
 319,628
 
 64.72
 
 4,878
 
 <img alt="The 12 Bezirke of Berlin" data-file-height="1151" data-file-width="1400" decoding="async" height="329" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Berlin%2C_administrative_divisions_%28%2Bdistricts_-boroughs_-pop%29_-_de_-_colored.svg/400px-Berlin%2C_administrative_divisions_%28%2Bdistricts_-boroughs_-pop%29_-_de_-_colored.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Berlin%2C_administrative_divisions_%28%2Bdistricts_-boroughs_-pop%29_-_de_-_colored.svg/600px-Berlin%2C_administrative_divisions_%28%2Bdistricts_-boroughs_-pop%29_-_de_-_colored.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Berlin%2C_administrative_divisions_%28%2Bdistricts_-boroughs_-pop%29_-_de_-_colored.svg/800px-Berlin%2C_administrative_divisions_%28%2Bdistricts_-boroughs_-pop%29_-_de_-_colored.svg.png 2x" width="400"/> 
 
 
 Friedrichshain-Kreuzberg 
 
 268,225
 
 20.16
 
 13,187
 
 
 Lichtenberg 
 
 259,881
 
 52.29
 
 4,952
 
 
 Marzahn-Hellersdorf 
 
 248,264
 
 61.74
 
 4,046
 
 
 Mitte 
 
 332,919
 
 39.47
 
 8,272
 
 
 Neukölln 
 
 310,283
 
 44.93
 
 6,804
 
 
 Pankow 
 
 366,441
 
 103.01
 
 3,476
 
 
 Reinickendorf 
 
 240,454
 
 89.46
 
 2,712
 
 
 Spandau 
 
 223,962
 
 91.91
 
 2,441
 
 
 Steglitz-Zehlendorf 
 
 293,989
 
 102.50
 
 2,818
 
 
 Tempelhof-Schöneberg 
 
 335,060
 
 53.09
 
 6,256
 
 
 Treptow-Köpenick 
 
 241,335
 
 168.42
 
 1,406

,Borough,Population 31 March 2010,Area in km²,Density per km²
0,Charlottenburg-Wilmersdorf,319628,64.72,4878
1,Friedrichshain-Kreuzberg,268225,20.16,13187
2,Lichtenberg,259881,52.29,4952
3,Marzahn-Hellersdorf,248264,61.74,4046
4,Mitte,332919,39.47,8272
5,Neukölln,310283,44.93,6804
6,Pankow,366441,103.01,3476
7,Reinickendorf,240454,89.46,2712
8,Spandau,223962,91.91,2441
9,Steglitz-Zehlendorf,293989,102.50,2818


In [35]:
CLIENT_ID = 'BYR4PKOI0IVYVC1RDPDVZEXJZRDYEAEZ10AUST3TEZMEWV4W' # my Foursquare ID
CLIENT_SECRET = 'XHQRWXL54LZ21GGDPNDY4YNODJOGZ3UC5CLYVJ5JQWXSEC1S' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BYR4PKOI0IVYVC1RDPDVZEXJZRDYEAEZ10AUST3TEZMEWV4W
CLIENT_SECRET:XHQRWXL54LZ21GGDPNDY4YNODJOGZ3UC5CLYVJ5JQWXSEC1S


In [36]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

52.5170365 13.3888599


In [37]:
from geopy.distance import vincenty
geolocator = Nominatim()

df['loc_coord'] = df['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [38]:
df

,Borough,Population 31 March 2010,Area in km²,Density per km²,loc_coord
0,Charlottenburg-Wilmersdorf,319628,64.72,4878,"(52.5078558, 13.2639518)"
1,Friedrichshain-Kreuzberg,268225,20.16,13187,"(52.5153063, 13.4616117)"
2,Lichtenberg,259881,52.29,4952,"(48.9212959, 7.4812268)"
3,Marzahn-Hellersdorf,248264,61.74,4046,"(52.5225225, 13.5876634)"
4,Mitte,332919,39.47,8272,"(52.5176896, 13.4023757)"
5,Neukölln,310283,44.93,6804,"(52.4811497, 13.4353501)"
6,Pankow,366441,103.01,3476,"(52.597636699999995, 13.436373975411648)"
7,Reinickendorf,240454,89.46,2712,"(52.6047631, 13.2952872)"
8,Spandau,223962,91.91,2441,"(52.535788, 13.1977924)"
9,Steglitz-Zehlendorf,293989,102.50,2818,"(52.4292052, 13.2299741)"


In [39]:
df[['Latitude', 'Longitude']] = df['loc_coord'].apply(pd.Series)
df

,Borough,Population 31 March 2010,Area in km²,Density per km²,loc_coord,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,319628,64.72,4878,"(52.5078558, 13.2639518)",52.507856,13.263952
1,Friedrichshain-Kreuzberg,268225,20.16,13187,"(52.5153063, 13.4616117)",52.515306,13.461612
2,Lichtenberg,259881,52.29,4952,"(48.9212959, 7.4812268)",48.921296,7.481227
3,Marzahn-Hellersdorf,248264,61.74,4046,"(52.5225225, 13.5876634)",52.522523,13.587663
4,Mitte,332919,39.47,8272,"(52.5176896, 13.4023757)",52.517690,13.402376
5,Neukölln,310283,44.93,6804,"(52.4811497, 13.4353501)",52.481150,13.435350
6,Pankow,366441,103.01,3476,"(52.597636699999995, 13.436373975411648)",52.597637,13.436374
7,Reinickendorf,240454,89.46,2712,"(52.6047631, 13.2952872)",52.604763,13.295287
8,Spandau,223962,91.91,2441,"(52.535788, 13.1977924)",52.535788,13.197792
9,Steglitz-Zehlendorf,293989,102.50,2818,"(52.4292052, 13.2299741)",52.429205,13.229974


In [40]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, population in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Population 31 March 2010']):
    label = '{}, {}'.format(population, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  
    
map_berlin

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
location_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Charlottenburg-Wilmersdorf
Friedrichshain-Kreuzberg
Lichtenberg
Marzahn-Hellersdorf
Mitte
Neukölln
Pankow
Reinickendorf
Spandau
Steglitz-Zehlendorf
Tempelhof-Schöneberg
Treptow-Köpenick


In [43]:
location_venues

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Charlottenburg-Wilmersdorf,52.507856,13.263952,Adik's Stehcafe,52.507889,13.258131,Café
1,Charlottenburg-Wilmersdorf,52.507856,13.263952,Die Wühlmäuse,52.508830,13.270733,Comedy Club
2,Charlottenburg-Wilmersdorf,52.507856,13.263952,Rasas,52.512100,13.264464,Indian Restaurant
3,Charlottenburg-Wilmersdorf,52.507856,13.263952,Hotel Villa Kastania,52.510310,13.268223,Hotel
4,Charlottenburg-Wilmersdorf,52.507856,13.263952,Rafih,52.508179,13.257651,Pizza Place
5,Charlottenburg-Wilmersdorf,52.507856,13.263952,Tutti Gusti,52.508860,13.268343,Italian Restaurant
6,Charlottenburg-Wilmersdorf,52.507856,13.263952,S Heerstraße,52.508120,13.259230,Light Rail Station
7,Charlottenburg-Wilmersdorf,52.507856,13.263952,H S Heerstraße,52.508610,13.258697,Bus Stop
8,Charlottenburg-Wilmersdorf,52.507856,13.263952,Il Passetto,52.508222,13.258438,Italian Restaurant
9,Friedrichshain-Kreuzberg,52.515306,13.461612,Casablanca,52.516890,13.461820,Falafel Restaurant


In [44]:
df1 = ((location_venues['Venue Category'] == 'Italian Restaurant').groupby(location_venues['Borough']).sum().astype(int).reset_index(name='Italian Restaurant'))
df1
df2 = pd.merge(df1,df,on='Borough')
df2

,Borough,Italian Restaurant,Population 31 March 2010,Area in km²,Density per km²,loc_coord,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,2,319628,64.72,4878,"(52.5078558, 13.2639518)",52.507856,13.263952
1,Friedrichshain-Kreuzberg,3,268225,20.16,13187,"(52.5153063, 13.4616117)",52.515306,13.461612
2,Lichtenberg,0,259881,52.29,4952,"(48.9212959, 7.4812268)",48.921296,7.481227
3,Marzahn-Hellersdorf,0,248264,61.74,4046,"(52.5225225, 13.5876634)",52.522523,13.587663
4,Mitte,1,332919,39.47,8272,"(52.5176896, 13.4023757)",52.517690,13.402376
5,Neukölln,3,310283,44.93,6804,"(52.4811497, 13.4353501)",52.481150,13.435350
6,Pankow,0,366441,103.01,3476,"(52.597636699999995, 13.436373975411648)",52.597637,13.436374
7,Reinickendorf,0,240454,89.46,2712,"(52.6047631, 13.2952872)",52.604763,13.295287
8,Spandau,1,223962,91.91,2441,"(52.535788, 13.1977924)",52.535788,13.197792
9,Steglitz-Zehlendorf,3,293989,102.50,2818,"(52.4292052, 13.2299741)",52.429205,13.229974


In [45]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
venues_onehot['Borough'] = location_venues['Borough'] 

# move borough column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Borough,ATM,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Costume Shop,Cupcake Shop,Department Store,Dessert Shop,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Garden,Gastropub,German Restaurant,Go Kart Track,Gourmet Shop,Grocery Store,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Industrial Estate,Italian Restaurant,Japanese Restaurant,Jazz Club,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Perfume Shop,Pet Store,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Road,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,South American Restaurant,Spa,Sports Bar,Sri Lankan Restaurant,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
berlin_grouped = venues_onehot.groupby('Borough').mean().reset_index()
berlin_grouped

,Borough,ATM,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Costume Shop,Cupcake Shop,Department Store,Dessert Shop,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Garden,Gastropub,German Restaurant,Go Kart Track,Gourmet Shop,Grocery Store,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Industrial Estate,Italian Restaurant,Japanese Restaurant,Jazz Club,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Perfume Shop,Pet Store,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Road,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,South American Restaurant,Spa,Sports Bar,Sri Lankan Restaurant,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,Charlottenburg-Wilmersdorf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.111111,0.000000,0.111111,0.000000,0.000000,0.222222,0.00,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0
1,Friedrichshain-Kreuzberg,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.110000,0.000000,0.010000,0.000000,0.01,0.000000,0.010000,0.010000,0.01,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.020000,0.000000,0.03,0.01,0.000000,0.010000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.0,0.01,0.010000,0.020000,0.030000,0.010000,0.000000,0.030000,0.01,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.01,0.070000,0.01,0.000000,0.000000,0.030000,0.01,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.01,0.000000,0.000000,0.020000,0.010000,0.020000,0.04,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.030000,0.030000,0.000000,0.01,0.020000,0.0
2,Lichtenberg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [47]:
berlin_grouped.shape

(12, 123)

In [48]:
num_top_venues = 5

for hood in berlin_grouped['Borough']:
    print("----"+hood+"----")
    temp = berlin_grouped[berlin_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Charlottenburg-Wilmersdorf----
                venue  freq
0  Italian Restaurant  0.22
1  Light Rail Station  0.11
2   Indian Restaurant  0.11
3         Comedy Club  0.11
4               Hotel  0.11


----Friedrichshain-Kreuzberg----
               venue  freq
0                Bar  0.11
1               Café  0.10
2        Pizza Place  0.07
3    Thai Restaurant  0.04
4  Indian Restaurant  0.03


----Lichtenberg----
           venue  freq
0         Hostel   0.5
1  Historic Site   0.5
2            ATM   0.0
3      Nightclub   0.0
4           Pool   0.0


----Marzahn-Hellersdorf----
                venue  freq
0         Supermarket  0.27
1                 ATM  0.09
2  Miscellaneous Shop  0.09
3           Drugstore  0.09
4               Plaza  0.09


----Mitte----
               venue  freq
0  German Restaurant  0.14
1             Museum  0.10
2     History Museum  0.08
3              Hotel  0.06
4        Art Gallery  0.06


----Neukölln----
                       venue  freq
0         

In [49]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [51]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Borough'] = berlin_grouped['Borough']

for ind in np.arange(berlin_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], num_top_venues)

In [52]:
venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Charlottenburg-Wilmersdorf,Italian Restaurant,Bus Stop,Indian Restaurant,Pizza Place,Hotel
1,Friedrichshain-Kreuzberg,Bar,Café,Pizza Place,Thai Restaurant,Falafel Restaurant
2,Lichtenberg,Hostel,Historic Site,Zoo Exhibit,Gastropub,Doner Restaurant
3,Marzahn-Hellersdorf,Supermarket,ATM,Trail,Doner Restaurant,Drugstore
4,Mitte,German Restaurant,Museum,History Museum,Art Gallery,Hotel
5,Neukölln,Bar,Café,Coffee Shop,Dive Bar,Bistro
6,Pankow,Tram Station,Asian Restaurant,Turkish Restaurant,Farmers Market,Fried Chicken Joint
7,Reinickendorf,Zoo Exhibit,Hostel,Liquor Store,Nature Preserve,Plaza
8,Spandau,Bakery,Drugstore,Clothing Store,Pub,Café
9,Steglitz-Zehlendorf,Italian Restaurant,Bus Stop,Liquor Store,Gastropub,Doner Restaurant


In [53]:
venues_sorted.shape

(12, 6)

In [54]:
berlin_grouped.shape

(12, 123)

In [55]:
kclusters = 4

berlin_grouped_clustering = berlin_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(berlin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_
berlin_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [56]:
berlin_grouped

,Cluster Labels,Borough,ATM,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Costume Shop,Cupcake Shop,Department Store,Dessert Shop,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Garden,Gastropub,German Restaurant,Go Kart Track,Gourmet Shop,Grocery Store,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Industrial Estate,Italian Restaurant,Japanese Restaurant,Jazz Club,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Perfume Shop,Pet Store,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Road,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,South American Restaurant,Spa,Sports Bar,Sri Lankan Restaurant,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,0,Charlottenburg-Wilmersdorf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.111111,0.000000,0.111111,0.000000,0.000000,0.222222,0.00,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0
1,0,Friedrichshain-Kreuzberg,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.110000,0.000000,0.010000,0.000000,0.01,0.000000,0.010000,0.010000,0.01,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.020000,0.000000,0.03,0.01,0.000000,0.010000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.0,0.01,0.010000,0.020000,0.030000,0.010000,0.000000,0.030000,0.01,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.01,0.070000,0.01,0.000000,0.000000,0.030000,0.01,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.01,0.000000,0.000000,0.020000,0.010000,0.020000,0.04,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.030000,0.030000,0.000000,0.01,0.020000,0.0
2,2,Lichtenberg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [57]:
final = pd.merge(venues_sorted,berlin_grouped,on='Borough')
final
final1 = pd.merge(df2,final, on = 'Borough')
final1

,Borough,Italian Restaurant_x,Population 31 March 2010,Area in km²,Density per km²,loc_coord,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels,ATM,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Costume Shop,Cupcake Shop,Department Store,Dessert Shop,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Garden,Gastropub,German Restaurant,Go Kart Track,Gourmet Shop,Grocery Store,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Industrial Estate,Italian Restaurant_y,Japanese Restaurant,Jazz Club,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Perfume Shop,Pet Store,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Road,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,South American Restaurant,Spa,Sports Bar,Sri Lankan Restaurant,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,Charlottenburg-Wilmersdorf,2,319628,64.72,4878,"(52.5078558, 13.2639518)",52.507856,13.263952,Italian Restaurant,Bus Stop,Indian Restaurant,Pizza Place,Hotel,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.111111,0.000000,0.111111,0.000000,0.000000,0.222222,0.00,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0
1,Friedrichshain-Kreuzberg,3,268225,20.16,13187,"(52.5153063, 13.4616117)",52.515306,13.461612,Bar,Café,Pizza Place,Thai Restaurant,Falafel Restaurant,0,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.110000,0.000000,0.010000,0.000000,0.01,0.000000,0.010000,0.010000,0.01,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.020000,0.000000,0.03,0.01,0.000000,0.010000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.0,0.01,0.010000,0.020000,0.030000,0.010000,0.000000,0.030000,0.01,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.01,0.070000,0.01,0.000000,0.000000,0.030000,0.01,0.000000,0.000000,0.0

In [58]:

# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final1['Latitude'], final1['Longitude'], final1['Borough'], final1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [59]:
final1.loc[final1['Cluster Labels'] == 0, final1.columns[[1] + list(range(5, final1.shape[1]))]].head()

,Italian Restaurant_x,loc_coord,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels,ATM,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Costume Shop,Cupcake Shop,Department Store,Dessert Shop,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Garden,Gastropub,German Restaurant,Go Kart Track,Gourmet Shop,Grocery Store,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Industrial Estate,Italian Restaurant_y,Japanese Restaurant,Jazz Club,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Perfume Shop,Pet Store,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Road,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,South American Restaurant,Spa,Sports Bar,Sri Lankan Restaurant,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,2,"(52.5078558, 13.2639518)",52.507856,13.263952,Italian Restaurant,Bus Stop,Indian Restaurant,Pizza Place,Hotel,0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.0,0.0,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.111111,0.000000,0.111111,0.000000,0.0,0.222222,0.00,0.000000,0.00,0.0,0.000000,0.111111,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.00,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0
1,3,"(52.5153063, 13.4616117)",52.515306,13.461612,Bar,Café,Pizza Place,Thai Restaurant,Falafel Restaurant,0,0.000000,0.000000,0.000000,0.01,0.020000,0.0,0.110000,0.0,0.01,0.000000,0.01,0.0,0.010000,0.010000,0.01,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.0,0.0,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.01,0.010000,0.010000,0.0,0.020000,0.0,0.03,0.01,0.0,0.010000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.0,0.010000,0.000000,0.000000,0.0,0.01,0.010000,0.020000,0.030000,0.010000,0.0,0.030000,0.01,0.000000,0.01,0.0,0.000000,0.000000,0.01,0.02,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.020000,0.000000,0.000000,0.01,0.000000,0.0,0.0,0.01,0.070000,0.01,0.000000,0.000000,0.030000,0.01,0.0,0.000000,0.01,0.0,0.000000,0.0,0.0,0.000000,0.01,0.0,0.01,0.01,0.000000,0.000000,0.020000,0.010000,0.02,0.04,0.000000,0.000000,0.000000,0.0,0.01,0.000000,0.030000,0.030000,0.000000,0.01,0.020000,0.0
3,0,"(52.5225225, 13.5876634)",52.522523,13.587663,Supermarket,ATM,Trail,Doner Restaurant,Drugstore,0,0.090909,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.0,0.000000,0.

In [60]:
final1.loc[final1['Cluster Labels'] == 1, final1.columns[[1] + list(range(5, final1.shape[1]))]].head()

,Italian Restaurant_x,loc_coord,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels,ATM,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Costume Shop,Cupcake Shop,Department Store,Dessert Shop,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Garden,Gastropub,German Restaurant,Go Kart Track,Gourmet Shop,Grocery Store,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Industrial Estate,Italian Restaurant_y,Japanese Restaurant,Jazz Club,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Perfume Shop,Pet Store,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Road,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,South American Restaurant,Spa,Sports Bar,Sri Lankan Restaurant,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
11,0,"(52.417893, 13.6001848)",52.417893,13.600185,Lake,Beach,Zoo Exhibit,Gastropub,Doner Restaurant,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
final1.loc[final1['Cluster Labels'] == 2, final1.columns[[1] + list(range(5, final1.shape[1]))]].head()

,Italian Restaurant_x,loc_coord,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels,ATM,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Costume Shop,Cupcake Shop,Department Store,Dessert Shop,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Garden,Gastropub,German Restaurant,Go Kart Track,Gourmet Shop,Grocery Store,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Industrial Estate,Italian Restaurant_y,Japanese Restaurant,Jazz Club,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Perfume Shop,Pet Store,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Road,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,South American Restaurant,Spa,Sports Bar,Sri Lankan Restaurant,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
2,0,"(48.9212959, 7.4812268)",48.921296,7.481227,Hostel,Historic Site,Zoo Exhibit,Gastropub,Doner Restaurant,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
final1.loc[final1['Cluster Labels'] == 3, final1.columns[[1] + list(range(5, final1.shape[1]))]].head()

,Italian Restaurant_x,loc_coord,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels,ATM,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Costume Shop,Cupcake Shop,Department Store,Dessert Shop,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fountain,French Restaurant,Fried Chicken Joint,Garden,Gastropub,German Restaurant,Go Kart Track,Gourmet Shop,Grocery Store,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Industrial Estate,Italian Restaurant_y,Japanese Restaurant,Jazz Club,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Perfume Shop,Pet Store,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Road,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,South American Restaurant,Spa,Sports Bar,Sri Lankan Restaurant,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
6,0,"(52.597636699999995, 13.436373975411648)",52.597637,13.436374,Tram Station,Asian Restaurant,Turkish Restaurant,Farmers Market,Fried Chicken Joint,3,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0
